In [1]:
import xgboost as xbg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import joblib

%matplotlib inline

In [6]:
def load_cls(fname):
    return np.array(pickle.load(open(fname,"rb")))

X_train_cls = load_cls("/scratch/pbanerj6/sml-dataset/X_train_cls.p")
X_val_cls = load_cls("/scratch/pbanerj6/sml-dataset/X_val_cls.p")
X_test_cls = load_cls("/scratch/pbanerj6/sml-dataset/X_test_cls.p")

In [7]:
y_train = load_cls("/scratch/pbanerj6/sml-dataset/y_train.p")
y_val = load_cls("/scratch/pbanerj6/sml-dataset/y_val.p")
y_test = load_cls("/scratch/pbanerj6/sml-dataset/y_test.p")

In [59]:
import lightgbm as lgb

In [60]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'false',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

In [58]:
gbm = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=31, max_depth=-1, learning_rate=0.1, n_estimators=100, subsample_for_bin=200000, objective=None, class_weight=None, min_split_gain=0.0, min_child_weight=0.001, min_child_samples=20, subsample=1.0, subsample_freq=0, colsample_bytree=1.0, reg_alpha=0.0, reg_lambda=0.0, random_state=None, n_jobs=-1, silent=True, importance_type='split',)

In [61]:
gbm.fit(X_train_cls,y_train,eval_set=[(X_val_cls,y_val)])

[1]	valid_0's binary_logloss: 0.688172
[2]	valid_0's binary_logloss: 0.683606
[3]	valid_0's binary_logloss: 0.679724
[4]	valid_0's binary_logloss: 0.675845
[5]	valid_0's binary_logloss: 0.671828
[6]	valid_0's binary_logloss: 0.668754
[7]	valid_0's binary_logloss: 0.666468
[8]	valid_0's binary_logloss: 0.664212
[9]	valid_0's binary_logloss: 0.661609
[10]	valid_0's binary_logloss: 0.660017
[11]	valid_0's binary_logloss: 0.658774
[12]	valid_0's binary_logloss: 0.656809
[13]	valid_0's binary_logloss: 0.655337
[14]	valid_0's binary_logloss: 0.653875
[15]	valid_0's binary_logloss: 0.652792
[16]	valid_0's binary_logloss: 0.651982
[17]	valid_0's binary_logloss: 0.650973
[18]	valid_0's binary_logloss: 0.650083
[19]	valid_0's binary_logloss: 0.649052
[20]	valid_0's binary_logloss: 0.647904
[21]	valid_0's binary_logloss: 0.646804
[22]	valid_0's binary_logloss: 0.645404
[23]	valid_0's binary_logloss: 0.644785
[24]	valid_0's binary_logloss: 0.64432
[25]	valid_0's binary_logloss: 0.643422
[26]	valid

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [62]:
y_pred = gbm.predict(X_test_cls)

In [63]:
from sklearn.metrics import accuracy_score

In [64]:
y_pred[1000]

1

In [65]:
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.79%


In [67]:
preds_test = gbm.predict(X_test_cls)
accuracy = accuracy_score(y_test, preds_test)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 61.79%


In [68]:
X_test_cls[0].shape[0]

768

In [73]:
from scipy.stats import rankdata

def mrrs(out, labels):
#     print(out,labels)
    outputs = np.argmax(out,axis=1)
    mrr = 0.0 
    for label,ranks in zip(labels,out):
        ranks = rankdata(ranks*-1)
        rank = ranks[label]
#         print(rank,ranks)
        mrr+=1/rank
    return mrr/len(labels)

def mrrwrapper(qid2c,qid2indexmap,preds_prob):
    labels = []
    out = []
    for qid in qid2c.keys():
        scores = []
        for ix in qid2indexmap[qid]:
            if len(scores) < 6:
                scores.append(preds_prob[ix][1])
        if len(scores) < 6:
            continue
        out.append(scores)
        labels.append(int(qid2c[qid]))
    return mrrs(np.array(out),labels)

def load_ranking(fname):
    return pickle.load(open("/scratch/pbanerj6/sml-dataset/ranking_"+fname+".p","rb"))

In [74]:
preds_test_probs = gbm.predict_proba(X_test_cls)

In [75]:
qid2c,qid2indexmap = load_ranking("test")

In [76]:
mrrwrapper(qid2c,qid2indexmap,preds_test_probs)

0.5735777777777804